# **Multi-Objective Generative Adversarial Network with 3D Dataset**

## Prepare necessary libraries and build dataset

In [ ]:
# ================== IMPORT LIBRARIES ==================
import os

# This allow a silent import of Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import time
import json
from statistics import mean
from google_drive_downloader import GoogleDriveDownloader

from IPython import display

print("Successfully Load All Libraries - Tensorflow Version {}".format(tf.__version__))

In [ ]:
# ================= SET UP PARAMETERS ==================

# The latent dimension of the generator network
latent_dimension = 200

# During the training, a number of samples using random latent values will be generated. This defines how many number of samples we want to generate
num_examples_to_generate = 25

# Epochs
epochs = 3000

# Batch size
batch_size = 32

# Learning rate
generator_learning_rate = 1e-4
realistic_discriminator_learning_rate = 1e-6

# Path of the directory where all the Json files are saved. 
data_files_directory = './data/chair_voxel_dataset/'

# Path of the directory where the pretrained weights of the stability auxiliary discriminator will be loaded from. 
stability_auxiliary_discriminator_load_location = './data/pretrained_models/chair_stability_auxiliary_discriminator/epoch_50'

# Path of the directory where the pretrained weights of the aesthetic auxiliary discriminator will be loaded from. 
aesthetic_auxiliary_discriminator_load_location = './data/pretrained_models/chair_aesthetic_auxiliary_discriminator/epoch_50'

# Path of the directory where the pretrained weights of the function auxiliary discriminator will be loaded from. 
function_auxiliary_discriminator_load_location = './data/pretrained_models/chair_function_auxiliary_discriminator/epoch_50'

# Path of the directory where the trained weights of the network will be saved. 
mo_gan_save_directory = './mogan_train/saved_models/'

# Create the directory to save the trained weights of the network. 
try:
    os.makedirs(mo_gan_save_directory)
except FileExistsError:
    print("The directory to save trained network weights already exists")

# The path of log file that contains training information. 
mo_gan_training_log_location = './mogan_train/logs.txt'

# Create a txt file to save losses during training
try:
    log_file = open(mo_gan_training_log_location, "x")
    log_file.close()
except FileExistsError:
    print("Log file already exists")
    
# During the training, a number of samples using random latent values will be generated. This defines where these generated samples will be saved
directory_to_generated_samples = './mogan_train/samples/'

try:
    os.makedirs(directory_to_generated_samples)
except FileExistsError:
    print("The directory to save generated samples already exists")

Download the dataset. 
Each file is a JSON file that contains information of the geometry we converted from the ShapeNet dataset and the related ratings. 

The JSON file should have following properties: 

*   "geometry":  a multi-dimension array to represent the geometry. 
*   "voxel_size": the size of each voxels (should always equal to 1 in the context of this notebook)
*   "scaling_factor_from_original": the scaling factor we used to scale the original mesh to produce this geometry
*   "geometry_shape": a list representing the shape of the geometry. (should always be [32, 32, 64] in the context of this notebook)

Ratings in the JSON files are as following: 

If the geometry is not given one of the ratings, the properties will note null
*   "stability_rating": the stability rating as a float from 0 to 1
*   "aesthetic_rating": the aesthetic rating as an integer from 0 to 10
*   "function_rating": the function rating as an integer from 0 to 10

Although the aesthetic_rating and function_rating are from 0 to 10 in the dataset. They should be normalized for the training, including the pretrained network should also produce a normalized rating. 

In [ ]:
# If the directory exists and is not empty we presume the dataset is already on the local drive. 
# If not, we will download the dataset. 

download_dataset_from_google_drive = False

if os.path.isdir(data_files_directory):
    if not os.listdir(data_files_directory):
        # Directory is empty
        download_dataset_from_google_drive = True
    else:    
        # Directory is not empty
        download_dataset_from_google_drive = False
else:
    # Given directory doesn't exist
    download_dataset_from_google_drive = True
    
if download_dataset_from_google_drive:

    # This will get download all the json files that contain geometry and rating information from google drive to the destination directory
    GoogleDriveDownloader.download_file_from_google_drive(file_id='1DJzSfSWUDkAhuEfu706pm0riC6ho_5NS', 
                                                          dest_path=os.path.join(data_files_directory, 'chair_voxel_dataset.zip'),
                                                          unzip=True)
    data_files_directory = data_files_directory + 'chair_voxel_dataset/'

In [ ]:
# Get all the json file paths from the unzippped directory. 
data_file_paths = glob.glob(data_files_directory + "*.json")

In [ ]:
data_files_with_stability_rating = []
data_files_with_aesthetic_rating = []
data_files_with_function_rating = []

for data_file in data_file_paths:
    with open(data_file) as json_file:
        dictionary = json.load(json_file)
        if dictionary["stability_rating"] != None:
            data_files_with_stability_rating.append(data_file)
        if dictionary["aesthetic_rating"] != None:
            data_files_with_aesthetic_rating.append(data_file)
        if dictionary["function_rating"] != None:
            data_files_with_function_rating.append(data_file)

In [ ]:
print("There are {} json files in total. ".format(len(data_file_paths)))
print("There are {} files that have stability rating. ".format(len(data_files_with_stability_rating)))
print("There are {} files that have aesthetic rating. ".format(len(data_files_with_aesthetic_rating)))
print("There are {} files that have function rating. ".format(len(data_files_with_function_rating)))

In [ ]:
# Define how many samples will be used for training
num_training_samples = 67  # 6777

Define several helper functions below that will facilitate building the tensorflow dataset. 

In [ ]:
# ================= HELPER FUNCTIONS ===================
# This function can preview a geometry data using matplot lib
def preview_geometries(geometry_data, plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None):
    # geometry_data must be an array of shape [32, 32, 64, 1]
    if stability_model is not None:
        stability_prediction = np.array(stability_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        stability_prediction = None
    
    if aesthetic_model is not None:
        aesthetic_prediction = np.array(aesthetic_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        aesthetic_prediction = None
    
    if function_model is not None:
        function_prediction = np.array(function_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        function_prediction = None
    
    generation_title = "Stability Prediction is {}\nAesthetic Prediction is {}\nFunction Prediction is {}".format(str(stability_prediction), str(aesthetic_prediction), str(function_prediction))

    x = []
    y = []
    z = []

    data = np.squeeze(geometry_data, axis=-1)
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            for k in range(0, len(data[i][j])):
                if data[i][j][k] >= plot_threshold:
                    x.append(i)
                    y.append(j)
                    z.append(k)

    z_c = z

    # We decided to plot the results with a gradient color map so the depth and geometries are clearer to see. 
    from matplotlib.colors import ListedColormap

    N = 256
    vals = np.ones((N, 4))
    vals[:, 0] = np.linspace(1, 0, N)
    vals[:, 1] = np.linspace(43/N, 43/N, N)
    vals[:, 2] = np.linspace(82/N, 82/N, N)
    custom_cmp = ListedColormap(vals)

    fig = plt.figure(figsize=(5,5))
    fig = plt.axes(projection='3d')
    
    # Data for three-dimensional scattered points
    fig.scatter3D(np.array(x), np.array(y) * -1, np.array(z), cmap=custom_cmp, c=z_c)
    fig.set_zlim(0,64)
    fig.set_ylim(-64,0)
    fig.set_xlim(0,64)
    plt.title(generation_title);
    if (save_plot_as_png):
        plt.savefig(path_to_save + '.png')
  
    plt.show()
    return plt

# This function reads a custom dictionary data
def read_json_file_geometry(filepath):
    with open(filepath.numpy()) as json_file:
        dictionary = json.load(json_file)

    tensor = tf.convert_to_tensor(np.array(dictionary["geometry"]), dtype=tf.float32)
    tensor = tf.expand_dims(tensor, -1)
    return tensor

# Function to build dataset without labels
def build_dataset_wo_labels(filepaths, batch_size):
    file_list = tf.data.Dataset.list_files(filepaths)
    ds = file_list.map(lambda x: tf.py_function(read_json_file_geometry, [x], Tout=tf.float32))
    ds = ds.shuffle(4, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(4)
    return ds

In [ ]:
# Build tensorflow dataset

train_ds = build_dataset_wo_labels(data_file_paths[0:num_training_samples], batch_size)

In [ ]:
# We can preview the first geometry in first batch as following

for each_batch in train_ds:
    preview_geometries(each_batch[0], plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None)
    break

## Load pretrained networks for multi-objective generative adversarial network training

Build the multi-objective generative adversarial network. 

In [ ]:
# ================ SET UP MACHINE LEARNING MODELS =========
def make_generator_model():
    model = tf.keras.Sequential()

    #Layer 1 - Fully Connected: Take in noise vector, use dense and reshape to 2x2x4x512
    model.add(tf.keras.layers.Dense(2*2*4*512, use_bias=False, input_shape=(latent_dimension,)))
    model.add(tf.keras.layers.Reshape((2, 2, 4, 512)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    assert model.output_shape == (None, 2, 2, 4, 512) 

    # Layer 2
    model.add(tf.keras.layers.Conv3DTranspose(256, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    assert model.output_shape == (None, 4, 4, 8, 256) 
    
    # Layer 3
    model.add(tf.keras.layers.Conv3DTranspose(128, (4,4,4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    assert model.output_shape == (None, 8, 8, 16, 128)

    # Layer 4
    model.add(tf.keras.layers.Conv3DTranspose(64, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    assert model.output_shape == (None, 16, 16, 32, 64)

    # Layer 5
    model.add(tf.keras.layers.Conv3DTranspose(1, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    assert model.output_shape == (None, 32, 32, 64, 1)
    model.add(tf.keras.layers.Activation('tanh'))

    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()

    #Layer 1
    model.add(tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha = 0.2))
    assert model.output_shape == (None, 16, 16, 32, 64)

    #Layer 2
    model.add(tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha =0.2 ))
    assert model.output_shape == (None, 8, 8, 16, 128)

    #Layer 3
    model.add(tf.keras.layers.Conv3D(256, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha = 0.2))
    assert model.output_shape == (None, 4, 4, 8, 256)

    #Layer 4
    model.add(tf.keras.layers.Conv3D(512, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha = 0.2))
    assert model.output_shape == (None, 2, 2, 4, 512)

    #Layer 6 Flatten and Dense
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [ ]:
generator = make_generator_model()
realistic_discriminator = make_discriminator_model()

In order to make sure the networks are set up correctly. 

Use random latent dimension values to generate a geometry and use the discriminator to make a prediction. 

In [ ]:
generated_geometry = generator(np.random.rand(1, latent_dimension))
realistic_discriminator(generated_geometry)

Build the stability auxiliary discriminator network and load the pretrained weights.

The weights to load must match structure as the network.

In [ ]:
stability_auxiliary_discriminator_model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
try:
    stability_auxiliary_discriminator_model.load_weights(stability_auxiliary_discriminator_load_location)
    
# If the pretrained weights cannot be loaded from the directory, we download the weights from google drive trained by researchers. 
except:
    GoogleDriveDownloader.download_file_from_google_drive(file_id='1bDwYinD8Tshf0GqnV-ktgRS8hDCUb1oZ', 
                                                          dest_path='./data/pretrained_models/chair_stability_auxiliary_discriminator.zip',
                                                          unzip=True)
    stability_auxiliary_discriminator_model.load_weights('./data/pretrained_models/chair_stability_auxiliary_discriminator/epoch_50')

print("Successfully loaded stability auxiliary discriminator weights")

Build the aesthetic auxiliary discriminator network and load the pretrained weights.

The weights to load must match structure as the network.

In [ ]:
aesthetic_auxiliary_discriminator_model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
try:
    aesthetic_auxiliary_discriminator_model.load_weights(aesthetic_auxiliary_discriminator_load_location)
    
# If the pretrained weights cannot be loaded from the directory, we download the weights from google drive trained by researchers. 
except:
    GoogleDriveDownloader.download_file_from_google_drive(file_id='1clRiK4mjHAi8jwR0F84HzEebtkhrryfA', 
                                                          dest_path='./data/pretrained_models/chair_aesthetic_auxiliary_discriminator.zip',
                                                          unzip=True)
    aesthetic_auxiliary_discriminator_model.load_weights('./data/pretrained_models/chair_aesthetic_auxiliary_discriminator/epoch_50')
    
print("Successfully loaded aesthetic auxiliary discriminator weights")

Build the function auxiliary discriminator network and load the pretrained weights.

The weights to load must match structure as the network.

In [ ]:
function_auxiliary_discriminator_model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 
   
    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 
   
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
try:
    function_auxiliary_discriminator_model.load_weights(function_auxiliary_discriminator_load_location)
    
# If the pretrained weights cannot be loaded from the directory, we download the weights from google drive trained by researchers. 
except:
    GoogleDriveDownloader.download_file_from_google_drive(file_id='1v2r9nfhA6HqHFZZ0xZ2W7u5X2pNF2eLp', 
                                                          dest_path='./data/pretrained_models/chair_function_auxiliary_discriminator.zip',
                                                          unzip=True)
    function_auxiliary_discriminator_model.load_weights('./data/pretrained_models/chair_function_auxiliary_discriminator/epoch_50')
    
print("Successfully loaded function auxiliary discriminator weights")

## Train the multi-objective generative adversarial network

Define the expectations for each category (the values are normalized)

Define the LAMBDA value (the weight) of each category, including the realistic adversarial network

Because these weights are relative to each other, it is recommended to experimente with these numbers. 

In [ ]:
use_realistic_discriminator = True  # use the realistic discriminator during MOGAN training

In [ ]:
stability_expectation = 0.05
aesthetic_expectation = 0.05
function_expectation = 0.85

In [ ]:
LAMBDA_realistic = 1.5
LAMBDA_obj = [1, 1, 1]  # [stability, aesthetic, function, realistic]

In [ ]:
log_file = open(mo_gan_training_log_location, "w")
log_file.write("Expectation Values for [stability, aesthetic, function]: {}".format([stability_expectation, aesthetic_expectation, function_expectation]))
log_file.write('\n')
log_file.write("Lambda Values for [stability, aesthetic, function, realistic]: {}".format(LAMBDA_obj))
log_file.write('\n')
log_file.write("Learning Rate for generator training: {}".format(generator_learning_rate))
log_file.write('\n')
log_file.write("Learning Rate for realistic discriminator training: {}".format(realistic_discriminator_learning_rate))
log_file.close()

Define the loss functions

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_real_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss, real_loss, fake_loss

In [ ]:
def generator_loss(generated_output, discriminator_label_obj, LAMBDA_obj, expectation_label_obj):
    
    generator_loss = cross_entropy(tf.ones_like(generated_output), generated_output)

    # objective discriminator loss using mean absolute error
    discriminator_obj_loss = tf.reduce_mean(tf.abs(discriminator_label_obj - expectation_label_obj), axis=0)

    if len(LAMBDA_obj) == 1:
        weighted_discriminator_obj_loss = LAMBDA_obj[0] * discriminator_obj_loss
    else:
        weighted_discriminator_obj_loss = tf.math.reduce_sum(tf.math.multiply(tf.cast(LAMBDA_obj, tf.float32), discriminator_obj_loss))

    weighted_disc_obj_losses_as_tensor = tf.math.multiply(tf.cast(LAMBDA_obj, tf.float32), discriminator_obj_loss)
    obj_1_loss = tf.math.reduce_mean(discriminator_obj_loss[0])
    obj_2_loss = tf.math.reduce_mean(discriminator_obj_loss[1])
    obj_3_loss = tf.math.reduce_mean(discriminator_obj_loss[2])
    
    if use_realistic_discriminator: 
        total_generator_loss = generator_loss * LAMBDA_realistic + weighted_discriminator_obj_loss
        return weighted_discriminator_obj_loss, obj_1_loss, obj_2_loss, obj_3_loss
    else:
        return total_generator_loss, obj_1_loss, obj_2_loss, obj_3_loss

In [ ]:
@tf.function
def mo_gan_train_step(geometries):
    noise = tf.random.normal([batch_size, latent_dimension])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generator output
        generated_geometry = generator(noise, training=True)
      
        # Discriminator output
        real_output = realistic_discriminator(geometries, training=True)
        fake_output = realistic_discriminator(generated_geometry, training=True)
      
        # Loss functions
        objective_stability = stability_auxiliary_discriminator_model(generated_geometry)
        expectation_stability = tf.ones_like(objective_stability) * stability_expectation
        objective_aesthetic = aesthetic_auxiliary_discriminator_model(generated_geometry)
        expectation_aesthetic = tf.ones_like(objective_aesthetic) * aesthetic_expectation
        objective_function = function_auxiliary_discriminator_model(generated_geometry)
        expectation_function = tf.ones_like(objective_function) * function_expectation
      
        objective_list = tf.concat([objective_stability, objective_aesthetic, objective_function], axis=-1)
        expectation_list = tf.concat([expectation_stability, expectation_aesthetic, expectation_function], axis=-1)

        gen_loss, loss_obj_stability, loss_obj_aesthetic, loss_obj_function = generator_loss(fake_output, objective_list, LAMBDA_obj, expectation_list)
        
        if use_realistic_discriminator:
            disc_loss, real_loss, fake_loss = discriminator_real_loss(real_output, fake_output)
    
    # Gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    if use_realistic_discriminator:
        gradients_of_discriminator = disc_tape.gradient(disc_loss, realistic_discriminator.trainable_variables)

    # Update both networks
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    if use_realistic_discriminator:
        realistic_discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, realistic_discriminator.trainable_variables))

    if use_realistic_discriminator:
        return gen_loss, disc_loss, loss_obj_stability, loss_obj_aesthetic, loss_obj_function
    else:
        return gen_loss, 0., loss_obj_stability, loss_obj_aesthetic, loss_obj_function

In [ ]:
# ---- Optimizers ----
generator_optimizer = tf.keras.optimizers.Adam(generator_learning_rate)
realistic_discriminator_optimizer = tf.keras.optimizers.Adam(realistic_discriminator_learning_rate)

In [ ]:
def write_sample_json(model, noise_input, directory_to_write, filename_to_write):
    predictions = model(noise_input)
    for i in range(predictions.shape[0]): # each num_examples_to_generate
        # Create the information to write
        dictionary_to_write = {
            "geometry" : np.array(predictions[i], dtype=np.float64).tolist(),
            "voxel_size" : 1,
            "scaling_factor_from_original" : None,
            "geometry_shape" : [32, 32, 64],

            # Could also include the discriminator predictions
            "stability_discriminator_prediction": np.array(stability_auxiliary_discriminator_model(tf.expand_dims(predictions[i], axis=0)), dtype=np.float64)[0][0],
            "aesthetic_discriminator_prediction": np.array(aesthetic_auxiliary_discriminator_model(tf.expand_dims(predictions[i], axis=0)), dtype=np.float64)[0][0], 
            "function_discriminator_prediction": np.array(function_auxiliary_discriminator_model(tf.expand_dims(predictions[i], axis=0)), dtype=np.float64)[0][0]
        }
        # we write dataset for each txt file
        with open(directory_to_write + filename_to_write + "_prediction_" + str(i) + ".json", 'w', encoding='utf-8') as f:
            json.dump(dictionary_to_write, f, ensure_ascii=False, indent=4)

In [ ]:
def mo_gan_train(train_dataset, epochs):
    
    noise_vector = tf.random.normal([num_examples_to_generate, latent_dimension], seed=101)

    for epoch in range(epochs):
        train_start = time.time()
        
        train_loss_generator = 0
        train_loss_realistic_discriminator = 0
        train_obj_1_losses = 0
        train_obj_2_losses = 0
        train_obj_3_losses = 0
        train_iters = 0
        
        for each_batch in train_dataset:
            loss_gen_loss, loss_disc_real, train_obj_1_loss, train_obj_2_loss, train_obj_3_loss = mo_gan_train_step(each_batch)
            train_obj_1_losses += train_obj_1_loss
            train_obj_2_losses += train_obj_2_loss
            train_obj_3_losses += train_obj_3_loss
            train_loss_generator += loss_gen_loss
            train_loss_realistic_discriminator += loss_disc_real
            train_iters += 1
            if train_iters == int(num_training_samples/batch_size):
                break
        
        log_file = open(mo_gan_training_log_location, "a")
        log_file.write('\n')
        log_file.write('\n')
        log_file.write("Epoch: " + str(epoch))
        log_file.write('\n')
        log_file.write("Generator training loss: {:0.5f}".format(train_loss_generator/train_iters))
        log_file.write('\n')
        log_file.write("Realistic discriminator training loss: {:0.5f}".format(train_loss_realistic_discriminator/train_iters))
        log_file.write('\n')
        log_file.write("Objective stability training loss: {:0.5f}".format(train_obj_1_losses/train_iters))
        log_file.write('\n')
        log_file.write("Objective aesthetic training loss: {:0.5f}".format(train_obj_2_losses/train_iters))
        log_file.write('\n')
        log_file.write("Objective function training loss: {:0.5f}".format(train_obj_3_losses/train_iters))
        log_file.close()

        write_sample_json(generator, noise_vector, directory_to_generated_samples, "epoch_" + str(epoch))
        
        display.clear_output(wait=False)
        
        print("Generated output: ")
        # Here we preview the geometry generated from the first of the noise vectors
        generated_geometry = np.squeeze(generator(np.expand_dims(noise_vector[0], axis=0)), axis=0)
        preview_geometries(generated_geometry, plot_threshold=0.25, stability_model=stability_auxiliary_discriminator_model, aesthetic_model=aesthetic_auxiliary_discriminator_model, function_model=function_auxiliary_discriminator_model)
        
        print('Epoch: {}, Generator training Loss: {:0.5f}, Realistic discriminator training loss: {:0.5f}, Time: {:0.1f}'.format(epoch, train_loss_generator/train_iters, train_loss_realistic_discriminator/train_iters, time.time() - train_start))

        # For every 10 epochs, we save the network weights
        if epoch % 10 == 0:
            generator.save_weights(mo_gan_save_directory + 'mo_vae_epoch_{}'.format(epoch))

In [ ]:
# mo_gan_train(train_ds, epochs)
mo_gan_train(train_ds, epochs)